In [159]:
from __future__ import division
from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import LabelEncoder

import torchvision.models as models
from multiprocessing import Pool
import time
import numpy as np
import pandas as pd
import datetime as dt
import os.path as dpath
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import pdb

import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

from IPython.display import FileLink, FileLinks


%matplotlib inline

In [160]:
path = "/home/edwin/Datasets/competitions/avito-demand-prediction/"

In [ ]:
!ls {path}

In [161]:
TRN_CSV = pd.read_csv(f"{path}train.csv")

In [162]:
TRN_CSV

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,Private,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,796.0,0.80323
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823.0,0.00000
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,Private,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,567.0,0.80323
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85,2017-03-17,Private,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,415.0,0.00000
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,Company,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,46.0,0.00000


In [ ]:
## Goal: Predict Deal Probability

In [ ]:
SAMPLE_SUBMIT = pd.read_csv(f"{path}sample_submission.csv")

In [ ]:
SAMPLE_SUBMIT

In [163]:
TEST_CSV = pd.read_csv(f"{path}test.csv")

In [154]:
len(TEST_CSV)

508438

In [ ]:
NEW_SUBMIT_CSV = pd.DataFrame()

In [ ]:
NEW_SUBMIT_CSV['item_id'] = TEST_CSV.item_id.values

In [ ]:
NEW_SUBMIT_CSV['deal_probability'] = np.zeros(len(TEST_CSV))

In [ ]:
NEW_SUBMIT_CSV

In [ ]:
NEW_SUBMIT_CSV.to_csv('new_submission.csv', index=False)

In [ ]:
FileLink('new_submission.csv')

In [176]:
class AvitoDataset(Dataset):
    def __init__(self, labels, root_dir, transform=None, test=False):
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.test = test
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image_name = self.labels.image.iloc[idx]
        full_name = f"{self.root_dir}{image_name}.jpg"
        try:
            image = Image.open(fullname).convert('RGB')
        except:
            img_zeros = np.zeros([224, 224, 3])
            image = Image.fromarray(img_zeros.astype('uint8'), 'RGB')
        if self.transform:
            image = self.transform(image)
        
        if self.test:
            return image
        else:
            label = self.labels.deal_probability.iloc[idx]
            return [image, label] 
            
            
        

In [48]:
ds_transform = transforms.Compose([transforms.Resize(224),
                                  transforms.CenterCrop(224),
                                  transforms.ToTensor()
                                 ])

In [49]:
avito_ds = AvitoDataset(TRN_CSV, data_dir, ds_transform)

In [12]:
data_dir = "/home/edwin/Datasets/competitions/avito-demand-prediction/data/competition_files/train_jpg/"

In [14]:
data_dir

'/home/edwin/Datasets/competitions/avito-demand-prediction/data/competition_files/train_jpg/'

In [50]:
avito_ds[0]

[
 ( 0 ,.,.) = 
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
      ...       ⋱       ...    
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
 
 ( 1 ,.,.) = 
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
      ...       ⋱       ...    
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
 
 ( 2 ,.,.) = 
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
      ...       ⋱       ...    
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
    0   0   0  ...    0   0   0
 [torch.FloatTensor of size 3x224x224], 0.12789]

In [51]:
train, valid = train_test_split(TRN_CSV, train_size=100, test_size=4, random_state=42)

In [53]:
len(train), len(valid)

(100, 4)

In [55]:
train_ds = AvitoDataset(train, data_dir, transform=ds_transform)
valid_ds = AvitoDataset(valid, data_dir, transform=ds_transform)

In [56]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=4)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=4)

In [57]:
use_gpu = torch.cuda.is_available()

In [128]:
class LinearModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        out = x.view(x.size(0),-1)
        out = self.linear(out)
        return out
    

In [131]:
input_dim = 150528
output_dim = 1
model = LinearModel(input_dim, output_dim)

In [132]:
model.cuda()

LinearModel(
  (linear): Linear(in_features=150528, out_features=1, bias=True)
)

In [62]:
criterion = nn.MSELoss()

In [64]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [67]:
inputs, labels = next(iter(train_dl))

In [70]:
if use_gpu:
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
else:
    inputs, labels = Variable(inputs), Variable(labels)

In [72]:
inputs.shape

torch.Size([64, 3, 224, 224])

In [75]:
d = inputs.view(inputs.size(0), -1).shape

In [88]:
inputs.view(-1,1)

Variable containing:
    0
    0
    0
  ⋮   
    0
    0
    0
[torch.cuda.FloatTensor of size 9633792x1 (GPU 0)]

In [90]:
linear = nn.Linear(1, 1)

In [92]:
linear(inputs.view(-1,1))

RuntimeError: Expected object of type Variable[torch.FloatTensor] but found type Variable[torch.cuda.FloatTensor] for argument #1 'mat1'

In [93]:

x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)


In [109]:
inp = Variable(torch.from_numpy(x_train).cuda())

In [110]:
inp.shape

torch.Size([11, 1])

In [129]:
inp2 = inputs.view(inputs.size(0),-1)

In [130]:
inp2.shape

torch.Size([64, 150528])

In [133]:
model(inp2)

Variable containing:
1.00000e-04 *
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
 -2.3660
[torch.cuda.FloatTensor of size 64x1 (GPU 0)]

In [135]:
labels

Variable containing:
 0.1572
 0.8032
 0.0000
 0.2102
 0.3397
 0.2271
 0.8032
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.4000
 0.8000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.6000
 0.8652
 0.0000
 0.0000
 0.0000
 0.0338
 0.0000
 0.0319
 0.1667
 0.0000
 0.7679
 0.1303
 0.0000
 0.0000
 0.0000
 0.0000
 0.0366
 0.2731
 0.0000
 0.0000
 0.8267
 0.0000
 0.0000
 0.0000
 0.0000
 0.3450
 0.7850
 0.0000
 0.1715
 0.0000
 0.0000
 0.1207
 0.1988
 0.0000
 0.0730
 0.0000
 0.0000
[torch.cuda.DoubleTensor of size 64 (GPU 0)]

In [182]:
epochs = 100
for epoch in range(epochs):
    epoch += 1
    for i, data in enumerate(train_dl, 0):
        inps, lbls = data
        if use_gpu:
            inputs, labels = Variable(inps.cuda()), Variable(lbls.cuda()).float()
        else:
            inputs, labels = Variable(inps), Variable(lbls).float()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1,1))
        loss.backward()
        optimizer.step()
        print('epoch {}, loss {}'.format(epoch, loss.data[0]))
        

epoch 1, loss 0.09586278349161148
epoch 1, loss 0.13488799333572388
epoch 2, loss 0.11512845754623413
epoch 2, loss 0.10063789784908295
epoch 3, loss 0.1269281506538391
epoch 3, loss 0.07966069132089615
epoch 4, loss 0.1365068405866623
epoch 4, loss 0.06263188272714615
epoch 5, loss 0.09727279096841812
epoch 5, loss 0.13238130509853363
epoch 6, loss 0.10016937553882599
epoch 6, loss 0.12723183631896973
epoch 7, loss 0.13634493947029114
epoch 7, loss 0.06291971355676651
epoch 8, loss 0.12830877304077148
epoch 8, loss 0.07720622420310974
epoch 9, loss 0.09096431732177734
epoch 9, loss 0.14359638094902039
epoch 10, loss 0.11866681277751923
epoch 10, loss 0.0943475067615509
epoch 11, loss 0.12286711484193802
epoch 11, loss 0.08688030391931534
epoch 12, loss 0.08591857552528381
epoch 12, loss 0.152566596865654
epoch 13, loss 0.12063853442668915
epoch 13, loss 0.09084221720695496
epoch 14, loss 0.10862420499324799
epoch 14, loss 0.112201027572155
epoch 15, loss 0.13421165943145752
epoch 15, 

In [150]:
len(model(inputs))

36

In [151]:
len(inputs)

36

In [158]:
TEST_CSV

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,6544e41a8817,dbe73ad6e4b5,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Отдам бесплатно,На ангарском,NaN,66,2017-04-18,Private,a8b57acb5ab304f9c331ac7a074219aed4d349d8aef386...,2020.0
1,65b9484d670f,2e11806abe57,Свердловская область,Нижняя Тура,Хобби и отдых,Велосипеды,Дорожные,NaN,NaN,Продам велосипед,"Продам велосипед KAMA F200,в нормальном состо...",3000.0,4,2017-04-16,Private,NaN,NaN
2,8bab230b2ecd,0b850bbebb10,Новосибирская область,Бердск,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,NaN,NaN,BBK,Продам новый телевизор BBK 32 диагональ смарт...,15000.0,15,2017-04-17,Private,8c361112cb049745ef2d1b0ae73594fc5c107286b0c942...,2960.0
3,8e348601fefc,5f1d5c3ce0da,Саратовская область,Саратов,Для дома и дачи,Бытовая техника,Для кухни,Вытяжки,NaN,Вытяжка Jetair 60,"Продам новую вытяжку в упаковке,с документами....",4500.0,70,2017-04-17,Private,NaN,NaN
4,8bd2fe400b89,23e2d97bfc7f,Оренбургская область,Бузулук,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Коляска зима-лето,Продам отличную коляску. б/у 1 год. все вопрос...,4900.0,15,2017-04-15,Private,bc3cf6deef10840fc302e38eb48fa7748aa1e28d534f8f...,1002.0
5,c63dbd6c657f,c2a632af2602,Ростовская область,Ростов-на-Дону,Для дома и дачи,Ремонт и строительство,Окна и балконы,NaN,NaN,Стекла для пластиковых окон,Продаю стеклопакеты новые:/\n1. 1322х652х24 /\...,500.0,39,2017-04-12,Private,7acdde494d48f4e46c8be158d64ad73a61d153b825c378...,1409.0
6,6d1a410df86e,b239811ad530,Оренбургская область,Оренбург,Бытовая электроника,Ноутбуки,NaN,NaN,NaN,Ноутбук Lenovo Ideapad 300-15Ibr арт.14899,Описание: /\nНоутбук Lenovo IdeaPad 300-15IBR ...,20990.0,57316,2017-04-17,Shop,f52abf9f68eca4aac35ed2b67bf093095950c417c080dd...,2961.0
7,e8d3e7922b80,d85fa02e6341,Калининградская область,Калининград,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки,NaN,Nillkin Phenom для Apple iPhone,Для iPhone 7 (черный)/\n /\nПродаются от магаз...,990.0,851,2017-04-18,Shop,2e2b5526345f5d66571615dd0fdc984f765a336ec6df3d...,2909.0
8,2bc1ab208462,ae6586719bec,Новосибирская область,Новосибирск,Бытовая электроника,Товары для компьютера,Флэшки и карты памяти,NaN,NaN,Флешка USB 32Gb smartbuy,новые/\n /\nесть синие и оранжевые/\n /\nSmart...,1200.0,10020,2017-04-18,Company,8aa1dfcbdc489750eb965e43ff545505f2c845990d73f9...,2815.0
9,7e05d77a9181,30ad26d633ef,Свердловская область,Полевской,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,23,Ботинки,"Фирма ""Кенгуру""",400.0,16,2017-04-12,Private,4870dbf90fffd7fcbc34ef30cd6c2cf3f2fbe81bc1ef66...,88.0


In [238]:
test_ds = AvitoDataset(TEST_CSV, data_dir, transform=ds_transform, test=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=4)

In [239]:
# predict test set
deal_probabilities = []
for i, inps in enumerate(test_dl, 0):
    if use_gpu:
        inputs = Variable(inps.cuda())
    else:
        inputs = Variable(inps)
    outputs = model(inputs)
    deal_probabilities.append(outputs.data.cpu())
deal_probability_final = np.concatenate(deal_probabilities).ravel()

In [240]:
len(deal_probability_final)

508438

In [241]:
len(TEST_CSV)

508438

In [242]:
deal_probability_final = np.clip(deal_probability_final, 0.025, 0.975)

In [243]:
# submit list
NEW_SUBMIT_CSV = pd.DataFrame()
NEW_SUBMIT_CSV['item_id'] = TEST_CSV.item_id.values
NEW_SUBMIT_CSV['deal_probability'] = deal_probability_final
NEW_SUBMIT_CSV.to_csv('new_submission.csv', index=False)
FileLink('new_submission.csv')

/home/edwin/Developer/iMaterial/avito-demand-prediction/new_submission.csv